<a href="https://colab.research.google.com/github/facebookresearch/vissl/blob/v0.1.6/tutorials/Feature_Extraction_V0_1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All rights reserved.

# Feature Extraction

In this tutorial, we look at a simple example of how to use VISSL to extract features after finished training the vissl moddels.

**EXAMPLE 1**: Download the pre-trained [jigsaw-retouch](https://drive.google.com/file/d/159SgjqklmLHWpEQNq14i_gJk0NDhyAHE/view?usp=sharing) to the `root` directory and rename it to `checkpoints_jigsaw_retouch`.

**EXAMPLE 2**: Download the pre-trained [jigsaw-cityscapes](https://drive.google.com/file/d/1Af710oLe_n1h4RMMnhdbxWQWiDCJx68j/view?usp=sharing) to the `root` directory and rename it to `checkpoints_jigsaw_cityscapes`.

VISSL should be successfuly installed by now and all the dependencies should be available.


In [2]:
import vissl
import tensorboard
import apex
import torch

/home/zeju/miniconda3/envs/vissl_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Using the custom data in VISSL

The original data is saved in the `data` directory. The transferred images are saved in such a way, that they are stored in the `data/generated_images/#epoch` directory (`#epoch` is the number of CycleGAN epoch).

**EXAMPLE 1**: download the retouch data set from [retouch-dataset](https://drive.google.com/file/d/1r8pQCoVzEAHdy9wLW_MUkyfgBBFePMPv/view?usp=sharing) and insert it into the `data/real_images` directory. Download the transferred images from [transferred-retouch-images](https://drive.google.com/file/d/1nMcyF-z2yvPBDY70qBsT2Ydg7NUITpmR/view?usp=sharing) and isert the subfolders with the epoch number into the `data/generated_images` directory.

**EXAMPLE 2**: download the truncated retouch GTAV data set from [gta5-truncated-dataset](https://drive.google.com/file/d/1R9zmrwAKf03KOq9MSfhdPd6xOVRGEtrY/view?usp=sharing) and insert it into the `data/real_images` directory. Download the transferred images from [transferred-gta5-images](https://drive.google.com/file/d/1SLdGNHDi3LZTHXXNMNFDTmAQibAEjj-x/view?usp=sharing) and isert the subfolders with the epoch number into the `data/generated_images` directory. Note, it also works with the whole data set, one only has to change the `splits/gta5.txt` to the whole dataset. The truncated version is used due to memory and time efficiency.

In [3]:
from omegaconf import OmegaConf
from vissl.utils.hydra_config import AttrDict
from vissl.models import build_model
from classy_vision.generic.util import load_checkpoint
from vissl.utils.checkpoint import init_model_from_consolidated_weights
from PIL import Image
import torchvision.transforms as transforms
import argparse

from datasets import create_dataset

import sys, os

from tqdm import tqdm

import numpy as np

import csv
import pandas as pd

from scipy import linalg
import torch

from datasets import BaseDataset

from datasets.cityscapes_dataset import CityscapesDataset
from datasets.gta_dataset import GTA5Dataset
from datasets.retouch_dataset import Retouch_dataset


def calculate_fid(feat1, feat2):
    """ Calculate FID between feature distribution 1 and feature distribution 2
    Args:
        feat1: np.array, shape: (N, 2048), dtype: torch.float32 in range 0 - 1
        feat2: np.array, shape: (N, 2048), dtype: torch.float32 in range 0 - 1
    Returns:
        FID (scalar)
    """
    mu1, sigma1 = calculate_activation_statistics(feat1)
    mu2, sigma2 = calculate_activation_statistics(feat2)
    fid = calculate_frechet_distance(mu1, sigma1, mu2, sigma2)

    return fid


def calculate_activation_statistics(feat):
    """Calculates the statistics used by FID
    Args:
        feat: torch.tensor, shape: (N, 2048), dtype: torch.float32 in range 0 - 1
    Returns:
        mu:     mean over all activations from the last pool layer of the inception model
        sigma:  covariance matrix over all activations from the last pool layer
                of the inception model.
    """

    feat_np = feat.cpu().detach().numpy()
    mu = np.mean(feat_np, axis=0) # (2048, 0)
    sigma = np.cov(feat_np, rowvar=False) # (2048, 2048)
    return mu, sigma


# Modified from: https://github.com/bioinf-jku/TTUR/blob/master/fid.py
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).

    Stable version by Dougal J. Sutherland.
    Params:
    -- mu1 : Numpy array containing the activations of the pool_3 layer of the
             inception net ( like returned by the function 'get_predictions')
             for generated samples.
    -- mu2   : The sample mean over activations of the pool_3 layer, precalcualted
               on an representive data set.
    -- sigma1: The covariance matrix over activations of the pool_3 layer for
               generated samples.
    -- sigma2: The covariance matrix over activations of the pool_3 layer,
               precalcualted on an representive data set.
    Returns:
    --   : The Frechet Distance.
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, "Training and test mean vectors have different lengths"
    assert sigma1.shape == sigma2.shape, "Training and test covariances have different dimensions"

    diff = mu1 - mu2
    # product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = "fid calculation produces singular product; adding %s to diagonal of cov estimates" % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError("Imaginary component {}".format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return diff.dot(diff) + np.trace(sigma1) + np.trace(sigma2) - 2 * tr_covmean


def create_dataset(dataset_mode, folder_name, sample_list, size):
    """
    -- dataset_mode : specify the data set to be load
    -- folder_name  : the path to the directory, which the data is stored
    -- sample_list  : path to the .txt split file 
    -- size         : data crop size
    Returns:
    --   : dataset
    """
    if dataset_mode == "retouch":
        dataset = Retouch_dataset(base_dir=folder_name, list_dir=sample_list, size = size)
    elif dataset_mode == "gta5":
        dataset = GTA5Dataset(root=folder_name, list_path=sample_list, crop_size=size, ignore_label=19)
    elif dataset_mode == "cityscapes":
        dataset = CityscapesDataset(root=folder_name, list_path=sample_list, crop_size=size, ignore_label=19)
    else:
        print("Unrecognized dataset!")
        sys.exit()
        
    return dataset


def run_eval(opt):
    real_dataset = create_dataset(opt.real_dataset_mode, opt.real_dir, opt.real_list, opt.crop_size)
    fake_dataset = create_dataset(opt.fake_dataset_mode, opt.fake_dir, opt.fake_list, opt.crop_size)

    real_loader = torch.utils.data.DataLoader(real_dataset,
                                         batch_size=opt.batch_size,
                                         shuffle=opt.shuffle,
                                         num_workers=opt.num_threads,
                                         pin_memory=True,
                                         drop_last=False)

    fake_loader = torch.utils.data.DataLoader(fake_dataset,
                                         batch_size=opt.batch_size,
                                         shuffle=opt.shuffle,
                                         num_workers=opt.num_threads,
                                         pin_memory=True,
                                         drop_last=False)
    
    model.cuda()
    # model.eval()
    feature_fake = 0
    for idx, input in enumerate(tqdm(fake_loader)):
        features = model(input['image'].cuda())
        feat = torch.flatten(features[0], start_dim=1)
        if idx == 0:
            feature_fake = torch.zeros((len(fake_loader), feat.shape[1]))
            feature_fake[idx, :] = feat
        else:
            feature_fake[idx, :] = feat

    feature_real = 0
    if not os.path.exists("real.pt"):
        for idx, input in enumerate(tqdm(real_loader)):
            features = model(input['image'].cuda())
            feat = torch.flatten(features[0], start_dim=1)
            if idx == 0:
                feature_real = torch.zeros((len(real_loader), feat.shape[1]))
                feature_real[idx, :] = feat
                # feature_target = feat
            else:
                feature_real[idx, :] = feat
                # feature_target = torch.cat((feature_target, feat), 0)
            
        torch.save(feature_real, 'real.pt')
    else:
        feature_real = torch.load('real.pt')

    fid = calculate_fid(feature_fake, feature_real)
    print("Epoch {}:".format(opt.load_epoch), "score", fid)

    csv_path = os.path.join(os.getcwd(), "results", "self_supervised_results_{}_{}.csv".format(opt.real_dataset_mode, opt.method))
   
    if os.path.isfile(csv_path):
        x = []
        value = []
        with open(csv_path, 'r') as csvfile:
            lines = csv.reader(csvfile, delimiter=',')
            for idx, row in enumerate(lines):
                if idx != 0:
                    x.append(row[0])
                    value.append(row[1])
        x.append(opt.load_epoch)
        value.append(fid)
        x_np = np.asarray(x).astype(int)
        value_np = np.asarray(value).astype(float)

    to_write = []
    to_write.append(["epoch", opt.method])

    if os.path.isfile(csv_path):
        for epoch in range(len(x_np)):
            result = [x_np[epoch], value_np[epoch]]
            to_write.append(result)
    else:
        result = [opt.load_epoch, fid]
        to_write.append(result)

    with open(csv_path, "w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(to_write)
DataLoader

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



Specify the correct config file:
```
jigsaw_custom_retouch
jigsaw_custom_cityscapes
jigsaw_custom_mnist
jigsaw_custom_synthia
rotnet_custom_retouch
rotnet_custom_cityscapes
rotnet_custom_mnist
rotnet_custom_synthia
```
Extract features and calculating the FID score using pre-trained self-supervised models. 

**EXAMPLE 1**: evaluate the transformation from GTAV to Cityscapes dataset.
Specify the `real_dataset_mode` as `cityscapes` and the `fake_dataset_mode` as `gta5`(transferred images). Specify the path to the original target data `real_dir` and the transfearred source data `fake_base_dir` and `fake_dir`. Split files are stored in the `splits` directory. The models are saved in the directory with pattern like `checkpoints_jigsaw_retouch`.

In [ ]:
import argparse
import os


if __name__ == '__main__':
    opt = argparse.ArgumentParser()
    opt.real_dataset_mode = "retouch"
    opt.fake_dataset_mode = "retouch"
    opt.method = "jigsaw"
    opt.real_dir = os.path.join(os.getcwd(), "datasets/real_images/retouch-dataset")
    opt.fake_base_dir = os.path.join(os.getcwd(), "datasets/generated_images/OCT_new")
    opt.fake_dir = os.path.join(os.getcwd(), "datasets/generated_images/OCT_new")
    opt.load_epoch = 0
    opt.model_phase = 90
    
    opt.real_list = os.path.join(os.getcwd(), "splits/spectralis_samples.txt")
    opt.fake_list = os.path.join(os.getcwd(), "splits/cirrus_samples.txt")
    
    opt.crop_size= (512, 512)
    
    opt.num_threads = 0  
    opt.batch_size = 1 
    opt.shuffle = True  
    opt.no_flip = True  
    opt.display_id = -1
    
    config = OmegaConf.load("configs/config/pretrain/{}/{}_custom_{}.yaml".format(opt.method, opt.method, opt.real_dataset_mode))

    default_config = OmegaConf.load("vissl/config/defaults.yaml")
    cfg = OmegaConf.merge(default_config, config)

    cfg = AttrDict(cfg)
    cfg.config.MODEL._MODEL_INIT_SEED = 0
    cfg.config.MODEL.WEIGHTS_INIT.PARAMS_FILE = "./checkpoints_{}_{}/model_phase{}.torch".format(opt.method, opt.real_dataset_mode, opt.model_phase)
    cfg.config.MODEL.FEATURE_EVAL_SETTINGS.EVAL_MODE_ON = True
    cfg.config.MODEL.FEATURE_EVAL_SETTINGS.FREEZE_TRUNK_ONLY = True
    cfg.config.MODEL.FEATURE_EVAL_SETTINGS.EXTRACT_TRUNK_FEATURES_ONLY = True
    cfg.config.MODEL.FEATURE_EVAL_SETTINGS.SHOULD_FLATTEN_FEATS = False
    cfg.config.MODEL.FEATURE_EVAL_SETTINGS.LINEAR_EVAL_FEAT_POOL_OPS_MAP = [["res5avg", ["Identity", []]]]

    model = build_model(cfg.config.MODEL, cfg.config.OPTIMIZER)
    weights = load_checkpoint(checkpoint_path=cfg.config.MODEL.WEIGHTS_INIT.PARAMS_FILE)
    model.cuda()

    init_model_from_consolidated_weights(
        config=cfg.config,
        model=model,
        state_dict=weights,
        state_dict_key_name="classy_state_dict",
        skip_layers=[],  # Use this if you do not want to load all layers
    )

    head = "results/"

    if not os.path.exists(head):
        os.makedirs(head)

    transferred_images_dir = opt.fake_dir
    epochs = [int(f) for f in os.listdir(transferred_images_dir) if os.path.isdir(os.path.join(transferred_images_dir, f))]
    epochs.sort()
    
    # target feature distribtuion
    if os.path.exists("real.pt"):
        os.remove("real.pt")

    for epoch in epochs:
        print("run eval epoch {}".format(epoch))
        opt.fake_dir = os.path.join(opt.fake_base_dir, "{}".format(epoch))
        opt.load_epoch = int(epoch)
        run_eval(opt)


run eval epoch 6


100%|███████████████████████████████████████| 1176/1176 [00:24<00:00, 47.29it/s]


Epoch 6: score 2.2903170215644884
run eval epoch 9


100%|███████████████████████████████████████| 3072/3072 [01:11<00:00, 43.26it/s]


Epoch 9: score 1.8162163244578187
run eval epoch 12


...

100%|███████████████████████████████████████| 3072/3072 [01:15<00:00, 40.88it/s]


Epoch 75: score 0.7800928812733101
run eval epoch 78


100%|███████████████████████████████████████| 3072/3072 [01:12<00:00, 42.09it/s]


Epoch 78: score 0.5301610752178698
run eval epoch 81


 52%|████████████████████▎                  | 1603/3072 [00:39<00:35, 41.02it/s]